In [1]:
import my_functions as myfun
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import pywt
import cv2
from skimage.filters import gabor
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr, pearsonr
from scipy.signal import convolve
from itertools import product
from sklearn.manifold import MDS
from scipy.io import loadmat
from numpy.linalg import lstsq
from scipy.stats import t

## Project Work 5: vRF Modeling

## Load previous data necessary for this assignment

In [4]:
# Load the NIfTI file
bold_path = "subj1/bold.nii.gz"
bold_img = nib.load(bold_path)

# Extract the data as a NumPy array
bold_data = bold_img.get_fdata()

# Load the labels into a pandas DataFrame
labels = pd.read_csv("subj1/labels.txt", sep=" ", header=0, names=["Condition", "Run"])

# Get unique conditions from labels
unique_conditions = [condition for condition in labels["Condition"].unique() if condition != "rest"]

# Create the design matrix
design_matrix = myfun.create_design_matrix(labels)

# Modify the convolved matrix
design_matrix_with_intercepts  = myfun.add_run_intercepts(design_matrix, labels)

# Load the HRF file
hrf_path = "hrf.mat"  
hrf_data = loadmat(hrf_path)
hrf_sampled = hrf_data.get("hrf_sampled", None).flatten()  # Downsampled HRF

convolved_matrix = myfun.convolve_conditions(design_matrix_with_intercepts, hrf_sampled)

# load design matrix as "convolved_matrix"
X = convolved_matrix.values
df = X.shape[0] - np.linalg.matrix_rank(X)

# Fit the GLM and get results
residuals, beta_maps = myfun.fit_glm_and_generate_beta_maps(bold_data, X)

In [5]:
# Load ROI masks
vt_mask_path = "subj1/mask4_vt.nii.gz"
face_mask_path = "subj1/mask8_face_vt.nii.gz"
house_mask_path = "subj1/mask8_house_vt.nii.gz"

vt_mask = nib.load(vt_mask_path).get_fdata() > 0  # Ventral Temporal ROI
face_mask = nib.load(face_mask_path).get_fdata() > 0  # Face ROI
house_mask = nib.load(house_mask_path).get_fdata() > 0  # House ROI

### Task: vRF modeling of fMRI data

- How would you start implementing a voxel receptive field model?
- The Haxby data is not optimal for vRF fitting. Why?

In [20]:
def extract_wavelet_features(image, wavelet='haar', level=2):
    """
    Extract wavelet features from an image.
    """
    coeffs = pywt.wavedec2(image, wavelet, level=level)
    features = []

    for coeff in coeffs:
        if isinstance(coeff, tuple):
            # Detail coefficients (horizontal, vertical, diagonal)
            for c in coeff:
                features.append(np.mean(c))
                features.append(np.std(c))
        else:
            # Approximation coefficients
            features.append(np.mean(coeff))
            features.append(np.std(coeff))

    return np.array(features)

In [36]:
def load_and_extract_wavelet_features(stimuli_folder, wavelet='haar', level=2):
    """
    Load all images from the stimuli folder and extract Wavelet features.
    """
    
    all_features = []
    categories = [d for d in os.listdir(stimuli_folder) if os.path.isdir(os.path.join(stimuli_folder, d))]

    for category in categories:
        category_path = os.path.join(stimuli_folder, category)
        for file_name in os.listdir(category_path):
            file_path = os.path.join(category_path, file_name)

            if not os.path.isfile(file_path):
                continue

            image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            
            if image is None:
                print(f"Warning: Failed to load image {file_path}")
                continue

            # Normalize
            image = image / 255.0
            features = extract_wavelet_features(image, wavelet, level)
            all_features.append(features)

    return np.array(all_features)

In [38]:
# Apply wavelet feature extraction
stimuli_folder = "stimuli"
wavelet_features = load_and_extract_wavelet_features(stimuli_folder)

print(f"Extracted wavelet features shape: {wavelet_features.shape}") 

Extracted wavelet features shape: (335, 14)


In [23]:
# Count occurrences of each condition in the labels file
condition_counts = labels["Condition"].value_counts()
print(condition_counts)

Condition
rest            588
scissors        108
face            108
cat             108
shoe            108
house           108
scrambledpix    108
bottle          108
chair           108
Name: count, dtype: int64


In [29]:
stimuli_folder = "stimuli"
controls_folder = os.path.join(stimuli_folder, "controls")

# Count total images
stimuli_count = sum([len(files) for _, _, files in os.walk(stimuli_folder) if "controls" not in _])
controls_count = sum([len(files) for _, _, files in os.walk(controls_folder)])

print(f"Total images in stimuli : {stimuli_count}")
print(f"Total images in controls: {controls_count}")

Total images in stimuli : 663
Total images in controls: 328


In [25]:
# Convolve Gabor features with HRF
def convolve_with_hrf(features, hrf):
    """
    Convolve features with the HRF.
    Args:
        features (numpy.ndarray): Design matrix (rows = images, cols = Gabor features).
        hrf (numpy.ndarray): Hemodynamic Response Function.
    Returns:
        convolved_features (numpy.ndarray): Convolved design matrix.
    """
    convolved_features = np.zeros_like(features)
    for col in range(features.shape[1]):  # Convolve each feature column
        convolved_features[:, col] = convolve(features[:, col], hrf, mode='full')[:features.shape[0]]
    return convolved_features

In [26]:
# Apply HRF convolution
convolved_design_matrix = convolve_with_hrf(wavelet_features, hrf_sampled)
print(f"Convolved design matrix shape: {convolved_design_matrix.shape}")  # Expected: (1452, 16)

Convolved design matrix shape: (335, 14)


In [27]:
def fit_vrf_model(voxel_data, design_matrix):
    beta_weights = np.linalg.pinv(design_matrix.T @ design_matrix) @ design_matrix.T @ voxel_data
    return beta_weights

In [28]:
# Extract voxel data from the ventral temporal ROI
vt_voxel_data = bold_data[vt_mask]  # Shape: (577, 1452)

# Fit the vRF Model
beta_weights = fit_vrf_model(vt_voxel_data.T, convolved_design_matrix)

print(f"Fitted vRF model. Beta weights shape: {beta_weights.shape}")  # Expected: (16, 577)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1452 is different from 335)